## GEI723 — Automne 2025
### Neurosciences computationnelles et applications en traitement de l'information

Contenu créé par Ismael Balafrej, Ph.D., Jean Rouat, Ph.D. et Ahmad El Ferdaoussi, Ph.D.

Ce notebook est une adaptation de la classification par STDP de MNIST par Diehl & Cook. L'article de Diehl & Cook est disponible ici: https://www.frontiersin.org/articles/10.3389/fncom.2015.00099/full

Le modèle est simplifié un peu et réécrit pour Brian2 et Python 3. Le code est à trous, et les parties à compléter sont indiquées par 'COMPLETER'.

ATTENTION: Dans l'exemple donné ici les auteurs utilisent seulement des données d'entrainement et de validation. Ils utilisent à tort le mot "test" pour désigner les données de validation.

Les équations et certains paramètres des neurones sont fournies.

Un exemple d'encodage par fréquence est utilisé (voir la partie "testons à présent le réseau entrainé'). Vous avez toute liberté d'utiliser un autre type d'encodage.

### Différences avec les travaux de Diehl et Cook et avec l'article
La solution python (Brian1) présentée à l'origine sur le github des auteurs de l'article ne reflète pas exactement les équations de l'article. Nous avons par ailleurs converti en Brian 2 une partie de ce code tout en ajoutant certains éléments qui ne sont pas toujours dans l'article (comme par exemple les connexions avec probabilité). À noter qu'une autre version traduite en Brian 2 existe sur le github: https://github.com/zxzhijia/Brian2STDPMNIST .
SVP ne tombez pas dans le piège de vouloir absolument reproduire de façon parfaite ces codes informatiques. Ce serait une perte de temps car vous serez interrogé sur votre compréhension de la matière. Passer beaucoup de temps à vouloir remplir un code à trous de façon parfaite ne serait pas intéressant si cette démarche ne vous permet pas de comprendre.

Méthode de travail recommandée:
- Inplémenter dans un premier temps votre solution selon la compréhension de l'article que vous avez sans vouloir "coller" exactement au code à trous. Ceci vous permettra de comprendre l'algorithme de STDP. Dans ce contexte, le code à trous (et les codes disponibles sur internet) sont des éléments pour améliorer votre compréhension et déverminer votre code et non pas pour les "copier" tels qu'ils sont.
- Ensuite seulement, vous pourrez inclure les variantes proposées dans l'article et/ou dans les codes d'exemples.

### Voici les différences et éléments principaux:
Le code vous laisse le choix:
 - de la façon d'initialiser les poids du réseau (vous pouvez ainsi comparer l'impact d'une initialisation aléatoire versus une fixe);
 - de la façon de présenter les entrées au réseau avec possibilité d'introduire des délais (aléatoires ou fixes) sur les neurones d'entrée;
 - de normaliser les poids (qui est différente de celle présentée à l'équation (3) de l'article). À noter que dans l'article, les auteurs disent avoir utilisé la normalisation divisive des poids selon les travaux de Goodhill et Barrow.
 - Créer des connexions avec une probabilité donnée, ce qui peut être utile pour la recherche de réseaux ayant le moins de paramètres possibles à stocker.
 - Possibilité de choisir le temps de présentation des données.
 - etc.

#### ATTENTION: Ce code est fourni à titre d'exemple. Vous n'avez aucune obligation de résoudre votre problème spécifiquement avec ce code.

In [ ]:
import numpy as np
from sklearn import datasets, model_selection
from brian2 import *
import matplotlib.pyplot as plt
%matplotlib inline

## Ensemble de données MNIST

On télécharge l'ensemble de données MNIST directement dans le code. MNIST est disponible à https://www.openml.org/d/554

L'argument 'data_home' peut être utilisé pour télécharger MNIST dans un répertoire de votre choix. (Le répertoire par défaut est '~/scikit_learn_data/'.)

In [ ]:
X_all, y_all = datasets.fetch_openml('mnist_784', version=1, return_X_y=True, data_home="data")

MNIST est constituté de 70k d'images de 28x28 pixels, de chiffres.

'X' est le vecteur d'images, et 'y' est le vecteur d'étiquettes (labels).

In [ ]:
X_all.shape, y_all.shape

La simulation avec toutes les images est assez longue à faire rouler. Utilisons uniquement un sous-ensemble de MNIST pour une simulation plus rapide.

In [ ]:
X = X_all[:10000]
y = y_all[:10000]

Divisons MNIST en ensembles d'entrainement et de test pour entrainer et tester le modèle post-apprentissage.

Si on travaille avec l'ensemble au complet, on peut prendre 10k pour le test par exemple.

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, y, test_size=100)

X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

Dessinons une image aléatoire de l'ensemble de données pour voir ce à quoi ressemble MNIST.

Les images dans MNIST sont des vecteurs. Il faut donc les ré-organiser en matrice 28x28 pour les afficher.

In [ ]:
index = np.random.randint(0, len(X_train)-1)

plt.figure()
plt.axis('off')
plt.imshow(X_train[index].reshape(28, 28), cmap=plt.cm.gray_r)
plt.title("Échantillon MNIST avec étiquette %s" % y_train[index]);

## Réseau de neurones Brian2

On peut maintenant créer un réseau Brian2. Commençons par définir quelques paramètres.

In [ ]:
# Fixons le seed aléatoire afin de pouvoir reproduire les résultats
np.random.seed(0)

# Horloge de Brian2
defaultclock.dt = 1 * units.ms

# Cible de génération de code pour Brian2
prefs.codegen.target = 'cython'

time_per_sample =   0.35 * units.second
resting_time = 0.15 * units.second

v_rest_e = -65. * units.mV
v_rest_i = -60. * units.mV

v_reset_e = -65. * units.mV
v_reset_i = -45. * units.mV

v_thresh_e = -52. * units.mV
v_thresh_i = -40. * units.mV

refrac_e = 5. * units.ms
refrac_i = 2. * units.ms

tc_theta = 1e7 * units.ms
theta_plus_e = 0.10 * units.mV

tc_pre_ee = 20 * units.ms
tc_post_1_ee = 20 * units.ms
tc_post_2_ee = 40 * units.ms

# Taux d'apprentissage
nu_ee_pre =  0.001
nu_ee_post = 0.05

Définissons l'entrée au réseau de neurones. L'entrée est un encodeur de type codage par fréquence (rate coding).

In [ ]:
input_group = PoissonGroup(784, rates=64*Hz)

Définissons notre modèle de neurone ainsi que nos groupes de neurones.

In [ ]:
neuron_model = '''
    dv/dt = ((v_rest_e - v) + (I_synE + I_synI) / nS) / tau  : volt (unless refractory)

    I_synE =  ge * nS * -v           : amp
    
    I_synI =  gi * nS * (d_I_synI-v) : amp
    
    dge/dt = -ge/(1.0*ms)            : 1
    
    dgi/dt = -gi/(2.0*ms)            : 1
    
    tau                              : second (constant, shared)

    d_I_synI                         : volt (constant, shared)

    dtheta/dt = -theta / (tc_theta)  : volt
'''


excitatory_group = NeuronGroup(
    N=100, model=neuron_model, refractory=refrac_e,
    threshold='v > v_thresh_e + theta', reset='v=v_reset_e; theta += theta_plus_e', method='euler')
excitatory_group.tau = 100 * units.ms
excitatory_group.d_I_synI = -100. * units.mV

inhibitory_group = NeuronGroup(
    N=100, model=neuron_model, refractory=refrac_i,
    threshold='v > v_thresh_i', reset='v=v_reset_i', method='euler')
inhibitory_group.tau = 10 * units.ms
inhibitory_group.d_I_synI = -85. * mV



Et les synapses.

In [ ]:
synapse_model = "w : 1"

stdp_synapse_model = '''
    w : 1

    plastic : boolean (shared) # Activer/désactiver la plasticité
    
    post2before : 1
    
    dpre/dt   =   -pre / tc_pre_ee : 1 (event-driven)
    
    dpost1/dt  = -post1 / tc_post_1_ee : 1 (event-driven)
    
    dpost2/dt = -post2 / tc_post_2_ee : 1 (event-driven)
'''

stdp_pre = '''
    ge_post += w
    
    pre = 1.
    
    w = clip(w + (nu_ee_pre * post1) * int(plastic), 0, 1)
'''

stdp_post = '''
    post2before = post2
    
    w = clip(w + (nu_ee_post * pre * post2before) * int(plastic), 0, 1)
    
    post1 = 1.
    
    post2 = 1.
'''

input_synapse = Synapses(input_group, excitatory_group, model=stdp_synapse_model, on_pre=stdp_pre, on_post=stdp_post)
input_synapse.connect(True) # Fully connected
input_synapse.delay = '0*ms'
input_synapse.plastic = True

# Xavier/Glorot-like initialization for SNNs
n_in = 784
n_out = 100
w_max = 0.25
input_synapse.w = f'rand() * {w_max * np.sqrt(2.0 / (n_in + n_out))}'

e_i_synapse = Synapses(excitatory_group, inhibitory_group, model=synapse_model, on_pre='ge_post += w')
e_i_synapse.connect(p=0.0025)
e_i_synapse.w = 'rand()*10.4'

i_e_synapse = Synapses(inhibitory_group, excitatory_group, model=synapse_model, on_pre='gi_post += w')
i_e_synapse.connect(p=0.9)
i_e_synapse.w = 'rand() * 17.0'

Combien de synapses a-t-on dans le réseau?

In [ ]:
print(len(input_synapse) + len(e_i_synapse) + len(i_e_synapse))

Définissons un 'readout' pour notre réseau.

In [ ]:
e_monitor = SpikeMonitor(excitatory_group, record=True)

Créons le réseau.

In [ ]:
net = Network(input_group, excitatory_group, inhibitory_group,
              input_synapse, e_i_synapse, i_e_synapse, e_monitor)

## Entrainement

Entrainons à présent notre réseau.

Créons une matrice 'spikes' pour assigner les étiquettes des classes post-apprentissage. Cette matrice accumulera le décompte de décharges par classe.

Notre readout 'e_monitor' est actif tout le long de la simulation avec toutes les images. Donc à chaque nouvelle présentation d'image, on doit soustraire l'ancien décompte de décharges. On utilisera le vecteur 'old_spike_counts' pour le faire.

In [ ]:
spikes = np.zeros((10, len(excitatory_group)))
old_spike_counts = np.zeros(len(excitatory_group))


In [ ]:
number_of_epochs = 1

for epoch in range(number_of_epochs):  # Fixed: renamed from i to epoch
    print('Starting iteration %i' % epoch)
    for sample_idx, (sample, label) in enumerate(zip(X_train, y_train)):  # Fixed: renamed from j to sample_idx
        # Afficher régulièrement l'état d'avancement
        if (sample_idx % 10) == 0:
            print("Running sample %i out of %i" % (sample_idx, len(X_train)))

        # Configurer le taux d'entrée
        input_group.rates = sample / 4 * units.Hz

        # Simuler le réseau
        net.run(time_per_sample)

        # Enregistrer les décharges
        spikes[int(label)] += e_monitor.count - old_spike_counts
        # Gardons une copie du décompte de décharges pour pouvoir calculer le prochain
        old_spike_counts = np.copy(e_monitor.count)

        # Arrêter l'entrée
        input_group.rates = 0 * units.Hz

        # Laisser les variables retourner à leurs valeurs de repos
        net.run(resting_time)

        # Normaliser les poids

        weight_matrix = np.zeros((len(input_group), len(excitatory_group)))
        weight_matrix[input_synapse.i, input_synapse.j] = input_synapse.w
        colSums = np.sum(weight_matrix, axis=0)
        colFactors = 120 / colSums
        for neuron_idx in range(len(excitatory_group)):
            weight_matrix[:, neuron_idx] *= colFactors[neuron_idx]
        input_synapse.w = weight_matrix[input_synapse.i, input_synapse.j]
        if sample_idx % 200 == 0:
            weight_matrix = np.zeros((784, len(excitatory_group)))
            weight_matrix[input_synapse.i, input_synapse.j] = input_synapse.w

            plt.figure(figsize=(10, 10))  # Larger figure for more neurons
            for i in range(min(100, len(excitatory_group))):  # Changed from 100 to 400
                plt.subplot(10, 10, i+1)  # Changed from 10x10 to 20x20 grid
                plt.imshow(weight_matrix[:, i].reshape(28, 28), cmap='gray')
                plt.axis('off')
            plt.tight_layout()
            plt.show()


## Test

Commençons par trouver le meilleur neurone pour chaque classe de MNIST.

In [ ]:
'''input_synapse.plastic = False  # Turn off learning
excitatory_group.theta = 0 * units.mV

assignment_spikes = np.zeros((10, len(excitatory_group)))
old_spike_counts = np.zeros(len(excitatory_group))

num_assignment_samples = 10  # per digit
for digit in range(10):

    # Get samples of this digit
    digit_indices = np.where(y_train == str(digit))[0][:num_assignment_samples]

    for idx in digit_indices:
        sample = X_train[idx]

        # Reset state for clean measurement
        excitatory_group.theta = 0 * units.mV

        input_group.rates = sample / 2 * units.Hz
        net.run(time_per_sample)

        # Count spikes for this digit
        assignment_spikes[digit] += e_monitor.count - old_spike_counts
        old_spike_counts = np.copy(e_monitor.count)

        input_group.rates = 0 * units.Hz
        net.run(resting_time)

# Assign each neuron to digit it fired most for
assignment_spikes[0] = assignment_spikes[0]*0.1*0.1
neuron_assignments = np.argmax(assignment_spikes, axis=0)
mostLabelNeuron = np.argmax(assignment_spikes, axis=1)'''

Testons à présent le réseau entrainé!

In [ ]:
labeled_neurons = np.argmax(spikes, axis=0)
print("Neurones assignés aux classes:", labeled_neurons)
from collections import Counter
counts = Counter(neuron_assignments)
for digit in range(10):
    print(f"Chiffre {digit}: {counts.get(digit, 0)} neurones")
# 1. Visualiser les poids
weight_matrix = np.zeros((784, len(excitatory_group)))
weight_matrix[input_synapse.i, input_synapse.j] = input_synapse.w

plt.figure(figsize=(10, 10))  # Larger figure for more neurons
for i in range(min(200, len(excitatory_group))):  # Changed from 100 to 400
    plt.subplot(10, 10, i+1)  # Changed from 10x10 to 20x20 grid
    plt.imshow(weight_matrix[:, i].reshape(28, 28), cmap='gray')
    #plt.title(f'N{i}: {neuron_assignments[i]}', fontsize=10, pad=2)
    plt.axis('off')
plt.tight_layout()
plt.show()

plt.imshow(weight_matrix[:28*28, :200], cmap='hot')
plt.colorbar()
plt.show()

In [ ]:
print(spikes.sum(axis=1))

In [ ]:
# Déasctiver la plasticité STDP
input_synapse.plastic = False

num_correct_output = 0

for i, (sample, label) in enumerate(zip(X_test, y_test)):
    # Afficher régulièrement l'état d'avancement
    if (i % 10) == 0:
        print("Running sample %i out of %i" % (i, len(X_test)))

    # Configurer le taux d'entrée
    # ATTENTION, vous pouvez utiliser un autre type d'encodage
    input_group.rates = sample / 4 * units.Hz

    # Simuler le réseau
    net.run(time_per_sample)

    # Calculer le nombre de décharges pour l'échantillon
    current_spike_count = e_monitor.count - old_spike_counts
    # Gardons une copie du décompte de décharges pour pouvoir calculer le prochain
    old_spike_counts = np.copy(e_monitor.count)

    class_responses = np.array([np.sum(current_spike_count[labeled_neurons == c])
                            for c in range(10)])
    output_label = np.argmax(class_responses)

    # Si la prédiction est correcte
    if output_label == int(label):
        num_correct_output += 1

    # Laisser les variables retourner à leurs valeurs de repos
    net.run(resting_time)


print("The model accuracy is : %.3f" % (num_correct_output / len(X_test)))